# Matrix Rank

This text and code is derived from Mike X Cohen's course on linear algebra. For more information, see https://www.udemy.com/linear-algebra-theory-and-implementation/?couponCode=LINALGPX7

# Table of contents
- [Terminology](#terms) 
- [Methods for Computing Rank](#methods)
- [Potential Problems](#problems)
- [Rank of Added and Multiplied Matrices](#addmult)
- [Rank of $A^T A$ and $ A A^T$](#ata)
- [Making a Matrix Full-rank by Shifting](#shift)
- [Determining if a Vector is in the Span of a Set](#span)

**Matrix Rank** - **single number** that tells us how much information is contained in the matrix, how much dimensions are encoded in the matrix. 

Notation:    
$r$ or $\text{rank}{(\mathbf{A})}$, $r \geq 0, r \in \mathbb{N}$

For a $M \times N$ matrix the rank is bounded by the smallest dimension:

$$\text{rank}(\mathbf{A}) <= \min(M,N)$$

<a id = "terms"></a>
## Terminology

For an $M \times M$ square matirx $\mathbf{A}$, if $\text{rank}{(\mathbf{A})} = M$, it's a **FULL RANK MATRIX**.

For an $M \times N$ matrix $\mathbf{A}$ with $M > N$ (more rows than columns), if $\text{rank}{(\mathbf{A})} = N$, it's a **FULL COLUMN RANK MATRIX**.

For an $M \times N$ matrix $\mathbf{A}$ with $M < N$ (more columns than than), if $\text{rank}{(\mathbf{A})} = M$, it's a **FULL ROW RANK MATRIX**.

For an $M \times N$ matrix $\mathbf{A}$, if $\text{rank}{(\mathbf{A})} < \min(M, N)$, it's **REDUCED RANK MATRIX** or **RANK DEFICIENT MATRIX** or **DEGENERATE MATRIX** or **LOW RANK MATRIX**.   
If this is a case for a square matrix, then it's a **SINGULAR MATRIX**.

The **RANK** of a matrix is the number of linearly independent columns , which is the same thing is the number of linearly independent rows. 

<a id = "methods"></a>
## Methods for Computing Rank

<ol>
<li>Compute the number of linearly independent columns:   
    <ul>
        <li> by visual inspection  </li>
        <li> by applying methods used to solve systems of simultaneous linear equations  </li>
    </ul>    
    </li>    
<li>Apply row reduction to reduce matrix to echelon form and count the number of pivots (non-efficient with large matrices) </li>
    <li>Compute the SVD and count the number of non-zero singular values   </li>
    <li>Compute the eigendecomposition and count the number of non-zero eigenvalues.   </li>
<ol>

<a id = "problems"></a>
## Potential Problems

### Computer rounding errors

When using method 4 for computing eigenvalues, it's not likely that we will get absolute zeros, so the question is what is the threshold for saying that we consider a value to be zero.   

To explore that a bit, I made an example. Let's say we have a $10 \times 10$ matrix of rank 8: 

In [2]:
import numpy as np

In [3]:
np.random.seed(42)
A = np.random.randint(-10,10,(10,10))
print("Current rank: " + str(np.linalg.matrix_rank(A)))
A[:,1]=A[:,3]
A[:,2]=A[:,3]
print("New rank: " + str(np.linalg.matrix_rank(A)))

Current rank: 10
New rank: 8


In this case, we expect that we have two zero eigenvalues (as described in method 4). And here is what we get: 

In [4]:
np.linalg.eigvals(A)

array([-2.02802199e+01+0.j        ,  8.38070692e+00+9.53708533j,
        8.38070692e+00-9.53708533j, -5.71886025e+00+9.52874321j,
       -5.71886025e+00-9.52874321j,  1.81057986e+00+5.41584672j,
        1.81057986e+00-5.41584672j, -6.64633119e-01+0.j        ,
       -2.19322383e-14+0.j        ,  3.25233600e-15+0.j        ])

Are -2.19322383e-14+0.j  and 3.25233600e-15+0.j zeros or not quite? 
As it has been said, the solution is setting a threshold.   
However, it migh be hard to choose one. 

### Noise  

In real-life applications, when a device is not calibrated perfectly, we can get noise in the data that might create an illusion that we have more dimensions encoded than we actually have. For example, we can think of the following matrix. In reality, it can be the case that rows 1 and 3 are linearly dependent, while because of the noise we still get a rank of 3: 

In [5]:
A = np.array([
    [1,0,0],
    [0,1,0],
    [1,0.000001,0.000001]
    
])

In [6]:
np.linalg.matrix_rank(A.T)

3

<a id = "addmult"></a>
## Rank of Added and Multiplied Matrices

We can't say for sure the rank of $\mathbf{A}+\mathbf{B}$ or $\mathbf{A} \mathbf{B}$, but we can set an upper boundary for it.

### Addition

For matrices $\mathbf{A} \in \mathbb{R}^{M \times N}$ and $\mathbf{B} \in \mathbb{R}^{M \times N}$:

$$\text{rank} (\mathbf{A} + \mathbf{B}) \leq \text{rank} (\mathbf{A})+\text{rank} (\mathbf{B})$$

Obviously, it is still true that $\text{rank} (\mathbf{A})+\text{rank} (\mathbf{B}) \leq \min(M,N)$

Rank is **invariant** to scalar multiplication: 

$$\text{rank} (\lambda \mathbf{A}) = \text{rank} (\mathbf{A})$$

$$\text{rank} (\lambda \mathbf{A} + \lambda \mathbf{B}) = \text{rank} (\mathbf{A} + \mathbf{B})$$

**Example**. Rank 2 matrix is added to rank 1 matrix. The resulting matrix has rank 3: 

$${\begin{bmatrix}
1&2&0\\
3&4&0\\
5&9&0\\
\end{bmatrix}} +  {\begin{bmatrix}
0&0&5\\
0&0&4\\
0&0&1\\
\end{bmatrix}} = {\begin{bmatrix}
1&2&5\\
3&4&4\\
5&9&1\\
\end{bmatrix}}$$

It's interesting to note that addition can also result in a **smaller rank** (e.g. one of the columns is zeroed out by cancellation) 

### Multiplication

$$\text{rank} (\mathbf{A} \mathbf{B}) \leq \min(\text{rank} (\mathbf{A}),\text{rank} (\mathbf{B}))$$

**Example**. Rank 2 matrix is right-multiplied by rank 1 matrix. The resulting matrix has rank 1: 



$${\begin{bmatrix}
1&2&0\\
3&4&0\\
5&9&0\\
\end{bmatrix}} \times  {\begin{bmatrix}
0&0&5\\
0&0&4\\
0&0&1\\
\end{bmatrix}} = {\begin{bmatrix}
0&0&13\\
0&0&31\\
0&0&61\\
\end{bmatrix}}$$

**Proof:**  
Let $\mathbf{AB} = \mathbf{C}$. We can represent it as: 

$$\mathbf{A} b_j = c_j$$

$c_j$ is a linear combination of columns (denoted as $b_j$) of $\mathbf{A}$, so the subspace, spanned by C is **at most** the same as spanned by A. Also, if $c_j$ is all 0s, then one column is out. 

$\implies \text{rank}(\mathbf{C}) \leq \text{rank}(\mathbf{A})$.

On the other hand, 

$a_i \mathbf{B} = c_i$ (row combination), so the subspace spanned by C is **at most** the same as spanned by B. 

$\implies  \text{rank}(\mathbf{C}) \leq \text{rank}(\mathbf{B})$

<a id = "ata"></a>
## Rank of $A^T A$ and $ A A^T$

$$\text{rank}(\mathbf{A}) = \text{rank}(\mathbf{A}^T \mathbf{A}) = \text{rank}(\mathbf{A}^T) = \text{rank}(\mathbf{A} \mathbf{A}^T)$$

### Explanation 1  

$$\mathbf{A}^T \mathbf{A} = \mathbf{C}$$

$\mathbf{A}^T a_j = c_j$ - combination of columns

Dimensionality is the same. Column $c_j$ is the combination of the columns of $\mathbf{A}^T \implies \text{rank}(\mathbf{C}) = \text{rank}(\mathbf{A}^T)$ 

### Explanation 2

$\mathbf{A} x = 0$ (x is in the nullspace of $\mathbf{A}$). Now we left-multiply both sides by $\mathbf{A}^T$:  

$$\mathbf{A}^T A x = 0$$

$\implies N(\mathbf{A}) = N(\mathbf{A}^T \mathbf{A})$ (same nullspace!)  

$\implies dim(N(\mathbf{A})) = dim(N(\mathbf{A}^T \mathbf{A}))$  

$\implies \text{rank}(\mathbf{A}) = \text{rank} (\mathbf{A}^T \mathbf{A})$

### Explanation 3 

Singular value decomposition

$$ A = \mathbf{ U \Sigma V^T}$$, where $\Sigma$ is the diagonal matrix of eigenvalues. 

$\mathbf(A^T A) = \mathbf{( U \Sigma V^T)}^T \mathbf{ U \Sigma V^T} = \mathbf{V \Sigma U^T U \Sigma V^T} = \mathbf{V \Sigma^2 V^T}$

Explanation: $\mathbf{\Sigma}$ is a diagonal matrix, so $\mathbf{\Sigma^T = \Sigma}$. $\mathbf{U}$ is upper triangular, so $\mathbf{U^T U} = I$, that's why it disappears from the equation. 

Now we need to count the number of non-zero singular values whcih is the same for $\Sigma$ and $\Sigma^2$ (since it's a diagonal matrix). It also has the same eigenvalues (however, squared).

$\implies \text{rank}(\mathbf{A}) = \text{rank} (\mathbf{A^T A})$

$\mathbf{A^T A}$ is **guaranteed** to be full rank if $\mathbf{A}$ is full rank.

**Example**. 

In [7]:
# set a sample matrix
A = np.random.randint(-10,10,(10,10))
print("Rank of A: " + str(np.linalg.matrix_rank(A)))
AtA = np.dot(np.transpose(A), A)
print("Rank of AtA: " + str(np.linalg.matrix_rank(AtA)))

Rank of A: 10
Rank of AtA: 10


## Scalar Multiplication and Rank

$$\text{rank} (\mathbf{A}) = \text{rank} (\lambda \mathbf{A}), \forall \lambda \neq 0$$

<a id = "shift"></a>
## Making a matrix full-rank by shifting

Rank-deficient matrices are the most common in practies and we often need to transform such matrices somehow to make them full rank. It's typicall done by regularization: 

$$\tilde {\mathbf{A}} = \mathbf{A} + \lambda \mathbf{I}$$

If $\mathbf{A}$ is square with reduced rank then we just add a fraction of identity matrix (that "has information in each column"): 

$${\begin{bmatrix}
1&1&4\\
2&2&7\\
3&3&11\\
\end{bmatrix}} + 0.001{\begin{bmatrix}
1&0&0\\
0&1&0\\
0&0&1\\
\end{bmatrix}} = 
{\begin{bmatrix}
1.001&1&4\\
2&2.001&7\\
3&3&11.001\\
\end{bmatrix}}$$ 

It's important to be carful with $\lambda$. E.g., what if $\mathbf{A} = -\lambda \mathbf{I}$? The goal is to make a matrix full-rank **without changing the information significantly!!!**. 

This technique is often called:   
- regularization  
- smoothing 
- inflating 

<a id="span"></a>
## Determining if a vector is in the span of a set

Let's say we have a vector $v$ and two sets: $S$ and $T$ and we need to define whether v is in their span.

Here is what I used to solve it. If a vector is in the span, then we can at least say that: 
- it can be "constructed" from the vectors in the set
- if we add it to the set, then the rank of the matrix, constructed of these vectors, will remain the same. 

So to check if it's in the span the easiest way is as follows: 
- construct a matrix from the set  
- calculate the rank of this matrix
- augment this matrix with the new vector  
- calculate the rank of the augmented matrix  
- compare the two ranks: if they are equal, then the vector is in the span

In [10]:
# sample vector
v = np.array([1,2,3,4])
# sample sets as matrices
S = np.array([[4,3,6,2], [0,4,0,1]])
T = np.array([[1,2,2,2], [0,0,1,2]])

# construct augmented matrices: 
Sandv = np.append(S,v).reshape(3,4).T
Tandv = np.append(T,v).reshape(3,4).T

In [12]:
# test if ranks are equal
print(np.linalg.matrix_rank(S) == np.linalg.matrix_rank(Sandv)) # False -> not in the span
print(np.linalg.matrix_rank(T) == np.linalg.matrix_rank(Tandv)) # True -> in the span

False
True
